# Azure AI-Agents mit Unterstützung für das Model Context Protocol (MCP)

Dieses Notebook zeigt, wie Azure AI-Agents mit den Tools des Model Context Protocol (MCP) verwendet werden können, um einen intelligenten Agenten zu erstellen, der externe MCP-Server für erweiterte Funktionen nutzen kann.


## Installieren Sie die erforderlichen NuGet-Pakete

Zunächst müssen wir das Azure AI Agents Persistent-Paket installieren, das die grundlegende Funktionalität für die Arbeit mit Azure AI Agents bereitstellt.


## Vorteile der schlüssellosen Authentifizierung

Dieses Notebook demonstriert die **schlüssellose Authentifizierung**, die mehrere Vorteile bietet:
- ✅ **Keine API-Schlüssel zu verwalten** - Nutzt Azure-Identitätsbasierte Authentifizierung
- ✅ **Erhöhte Sicherheit** - Keine Geheimnisse im Code oder in der Konfiguration gespeichert
- ✅ **Automatische Anmeldeinformationen-Rotation** - Azure verwaltet den Lebenszyklus der Anmeldeinformationen
- ✅ **Rollenbasierter Zugriff** - Nutzt Azure RBAC für fein abgestufte Berechtigungen

`DefaultAzureCredential` verwendet automatisch die beste verfügbare Quelle für Anmeldeinformationen:
1. Managed Identity (bei Ausführung in Azure)
2. Azure CLI-Anmeldeinformationen (während der Entwicklung)
3. Visual Studio-Anmeldeinformationen
4. Umgebungsvariablen (falls konfiguriert)


In [1]:
#r "nuget: Azure.AI.Agents.Persistent, 1.1.0-beta.4"

Installed Packages Azure.AI.Agents.Persistent, 1.1.0-beta.4

Installieren Sie das Azure Identity-Paket für die Authentifizierung mit Azure-Diensten mithilfe von DefaultAzureCredential.


In [2]:
#r "nuget: Azure.Identity, 1.14.2"

Installed Packages Azure.Identity, 1.14.2

## Erforderliche Namespaces importieren

Importieren Sie die notwendigen Namespaces für Azure AI Agents und Azure Identity.


In [3]:
using Azure.AI.Agents.Persistent;
using Azure.Identity;

## Konfigurieren des Azure AI Agent Client (Schlüssellose Authentifizierung)

Richten Sie die Konfigurationsvariablen ein und erstellen Sie den PersistentAgentsClient mithilfe der **schlüssellosen Authentifizierung**:
- **projectEndpoint**: Der Azure AI Foundry-Projektendpunkt
- **modelDeploymentName**: Der Name des bereitgestellten KI-Modells (GPT-4.1 nano)
- **mcpServerUrl**: Die URL des MCP-Servers (Microsoft Learn API)
- **mcpServerLabel**: Ein Label zur Identifizierung des MCP-Servers
- **DefaultAzureCredential**: Verwendet verwaltete Identitäten, Azure CLI oder andere Anmeldeinformationen (keine API-Schlüssel erforderlich)


In [ ]:
var projectEndpoint = "Your Azure AI Foundry Project Endpoint";
var modelDeploymentName = "Your Azure OpenAI Model Deployment Name";
var mcpServerUrl = "https://learn.microsoft.com/api/mcp";
var mcpServerLabel = "mslearn";
PersistentAgentsClient agentClient = new(projectEndpoint, new DefaultAzureCredential());

## Erstellen einer MCP-Tooldefinition

Erstellen Sie eine MCP-Tooldefinition, die eine Verbindung zum Microsoft Learn MCP-Server herstellt. Dadurch kann der Agent auf Inhalte und Dokumentationen von Microsoft Learn zugreifen.


In [ ]:
MCPToolDefinition mcpTool = new(mcpServerLabel, mcpServerUrl);

## Erstellen Sie den KI-Agenten

Erstellen Sie einen persistenten KI-Agenten mit dem angegebenen Modell und den MCP-Tools. Der Agent ist konfiguriert mit:
- Dem GPT-4.1 nano Modell
- Anweisungen zur Nutzung der MCP-Tools zur Unterstützung
- Zugriff auf den Microsoft Learn MCP-Server


In [ ]:
PersistentAgent agent = await agentClient.Administration.CreateAgentAsync(
   model: modelDeploymentName,
   name: "my-learn-agent",
   instructions: "You are a helpful agent that can use MCP tools to assist users. Use the available MCP tools to answer questions and perform tasks.",
   tools: [mcpTool]
   );

## Erstelle einen Thread und sende eine Nachricht

Erstelle einen Konversationsthread und sende eine Benutzeranfrage, die nach dem Unterschied zwischen Azure OpenAI und OpenAI fragt. Dies wird die Fähigkeit des Agenten testen, die MCP-Tools zu nutzen, um genaue Informationen bereitzustellen.


In [7]:
PersistentAgentThread thread = await agentClient.Threads.CreateThreadAsync();

// Create message to thread
PersistentThreadMessage message = await agentClient.Messages.CreateMessageAsync(
    thread.Id,
    MessageRole.User,
    "What's difference between Azure OpenAI and OpenAI?");

## Konfigurieren von MCP-Tool-Ressourcen (Schlüssellos)

Richten Sie die MCP-Tool-Ressourcen ein. Für einen vollständig schlüssellosen Ansatz können Sie benutzerdefinierte Header entfernen, wenn der MCP-Server die auf Azure-Identität basierende Authentifizierung unterstützt. Das folgende Beispiel zeigt, wie Header hinzugefügt werden können, falls erforderlich, aber für schlüssellose Szenarien sind diese möglicherweise nicht notwendig.


In [ ]:
// Option 1: Completely keyless (if MCP server supports Azure identity)
MCPToolResource mcpToolResource = new(mcpServerLabel);
ToolResources toolResources = mcpToolResource.ToToolResources();

// Option 2: With custom headers (if still needed for specific MCP servers)
// MCPToolResource mcpToolResource = new(mcpServerLabel);
// mcpToolResource.UpdateHeader("Authorization", "Bearer <your-token>");
// ToolResources toolResources = mcpToolResource.ToToolResources();

## Agent-Ausführung starten

Erstellen und starten Sie eine Ausführung, um die Nachricht des Benutzers zu verarbeiten. Der Agent wird die konfigurierten MCP-Tools und Ressourcen verwenden, um eine Antwort zu generieren.


In [9]:
ThreadRun run = await agentClient.Runs.CreateRunAsync(thread, agent, toolResources);

## Überwachen des Ablaufs und Bearbeiten von Tool-Genehmigungen (Keyless)

Überwachen Sie den Status des Agentenlaufs und bearbeiten Sie erforderliche Tool-Genehmigungen. Diese Schleife:
1. Wartet, bis der Ablauf abgeschlossen ist oder eine Aktion erforderlich ist
2. Genehmigt automatisch MCP-Tool-Aufrufe, wenn dies erforderlich ist
3. Bei schlüsselloser Authentifizierung sind Header möglicherweise nicht erforderlich, wenn der MCP-Server Azure-Identität unterstützt


In [ ]:
while (run.Status == RunStatus.Queued || run.Status == RunStatus.InProgress || run.Status == RunStatus.RequiresAction)
{
    await Task.Delay(TimeSpan.FromMilliseconds(1000));
    run = await agentClient.Runs.GetRunAsync(thread.Id, run.Id);

    if (run.Status == RunStatus.RequiresAction && run.RequiredAction is SubmitToolApprovalAction toolApprovalAction)
    {
        var toolApprovals = new List<ToolApproval>();
        foreach (var toolCall in toolApprovalAction.SubmitToolApproval.ToolCalls)
        {
            if (toolCall is RequiredMcpToolCall mcpToolCall)
            {
                Console.WriteLine($"Approving MCP tool call: {mcpToolCall.Name}");
                
                // Option 1: Keyless approval (no headers needed)
                toolApprovals.Add(new ToolApproval(mcpToolCall.Id, approve: true));
                
                // Option 2: With headers (if required by specific MCP server)
                // toolApprovals.Add(new ToolApproval(mcpToolCall.Id, approve: true)
                // {
                //     Headers = { ["Authorization"] = "Bearer <your-token>" }
                // });
            }
        }

        if (toolApprovals.Count > 0)
        {
            run = await agentClient.Runs.SubmitToolOutputsToRunAsync(thread.Id, run.Id, toolApprovals: toolApprovals);
        }
    }
}

Approving MCP tool call: microsoft_docs_search


## Konversationsergebnisse anzeigen

Rufen Sie alle Nachrichten im Thread ab und zeigen Sie sowohl die Frage des Benutzers als auch die Antwort des Agenten an. Die Nachrichten werden in chronologischer Reihenfolge mit Zeitstempeln und Rollenindikatoren angezeigt.


In [12]:
using Azure;

AsyncPageable<PersistentThreadMessage> messages = agentClient.Messages.GetMessagesAsync(
    threadId: thread.Id,
    order: ListSortOrder.Ascending
);

await foreach (PersistentThreadMessage threadMessage in messages)
{
    Console.Write($"{threadMessage.CreatedAt:yyyy-MM-dd HH:mm:ss} - {threadMessage.Role,10}: ");
    foreach (MessageContent contentItem in threadMessage.ContentItems)
    {
        if (contentItem is MessageTextContent textItem)
        {
            Console.Write(textItem.Text);
        }
        else if (contentItem is MessageImageFileContent imageFileItem)
        {
            Console.Write($"<image from ID: {imageFileItem.FileId}>");
        }
        Console.WriteLine();
    }
}

2025-07-16 06:39:43 -       user: What's difference between Azure OpenAI and OpenAI?
2025-07-16 06:39:51 -  assistant: The main difference between Azure OpenAI and OpenAI lies in their deployment, management, and integration options:

1. **Azure OpenAI**:
   - A cloud service offered through Microsoft Azure.
   - Provides access to OpenAI models with additional enterprise features like security, compliance, and scale.
   - Allows integration with other Azure services, enabling seamless use within existing Azure-based solutions.
   - Offers managed deployment, monitoring, and support within the Azure ecosystem.
   - Suitable for organizations looking for enterprise-grade security, compliance, and regional availability.

2. **OpenAI (OpenAI API)**:
   - A standalone API service provided directly by OpenAI.
   - Accessible via the OpenAI platform without the need for Azure.
   - Focused on providing GPT models, DALL-E, etc., primarily for developers and researchers.
   - Suitable for indi


---

**Haftungsausschluss**:  
Dieses Dokument wurde mit dem KI-Übersetzungsdienst [Co-op Translator](https://github.com/Azure/co-op-translator) übersetzt. Obwohl wir uns um Genauigkeit bemühen, beachten Sie bitte, dass automatisierte Übersetzungen Fehler oder Ungenauigkeiten enthalten können. Das Originaldokument in seiner ursprünglichen Sprache sollte als maßgebliche Quelle betrachtet werden. Für kritische Informationen wird eine professionelle menschliche Übersetzung empfohlen. Wir übernehmen keine Haftung für Missverständnisse oder Fehlinterpretationen, die sich aus der Nutzung dieser Übersetzung ergeben.
